In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!scp '/content/drive/My Drive/DataFiles/Pneumonia_Sample.zip' '/content/data.zip'
!unzip '/content/data.zip'  -d '/content/'

Archive:  /content/data.zip
replace /content/Pneumonia_Sample/test/Normal/IM-0122-0001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_dir = '/content/Pneumonia_Sample/train'

In [ ]:
image_generator = ImageDataGenerator(
                    rotation_range = 20,
                    width_shift_range = 0.1,
                    shear_range = 0.1,
                    zoom_range = 0.1,
                    fill_mode = 'nearest',
                    )

In [ ]:
train  = image_generator.flow_from_directory(
    train_dir,
    target_size = (180, 180),
    shuffle = True,
    batch_size = 8,
    class_mode = 'binary'
)

Found 194 images belonging to 2 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from keras.applications import VGG16


In [ ]:
VGG16_base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (180, 180, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
VGG16_base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 180, 180, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 180, 180, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 90, 90, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 90, 90, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 90, 90, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 45, 45, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 45, 45, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 45, 45, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 45, 45, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 22, 22, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 22, 22, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 22, 22, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 11, 11, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 11, 11, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 11, 11, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 11, 11, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 5, 5, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
VGG16_model = tf.keras.Sequential([
    VGG16_base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation = 'relu'),
    Dropout(0.5),
    Dense(128, activation="relu"),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64,activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1,activation="sigmoid")
    ])


In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
METRICS = ['accuracy',
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name = 'recall')]

VGG16_model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = METRICS)



In [ ]:
history = VGG16_model.fit(train, epochs = 10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 295s 11s/step - accuracy: 0.4925 - loss: 0.9723 - precision: 0.5641 - recall: 0.5057
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 314s 11s/step - accuracy: 0.3958 - loss: 1.0921 - precision: 0.4626 - recall: 0.4730
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 284s 11s/step - accuracy: 0.4506 - loss: 1.0899 - precision: 0.5542 - recall: 0.5059
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 277s 11s/step - accuracy: 0.6863 - loss: 0.6287 - precision: 0.7801 - recall: 0.6560
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 286s 11s/step - accuracy: 0.7165 - loss: 0.5795 - precision: 0.7639 - recall: 0.7090
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 279s 11s/step - accuracy: 0.8259 - loss: 0.4335 - precision: 0.8634 - recall: 0.8258
Epoch 7/10
 6/25 ━━━━━━━━━━━━━━━━━━━━ 3:36 11s/step - accuracy: 0.8306 - loss: 0.4821 - precision: 0.9166 - recall: 0.7950

KeyboardInterrupt: 